In [3]:
import matplotlib.pyplot as plt
from scipy.io import wavfile as wav
from scipy.fftpack import fft
import numpy as np

### Reading the audio files

In [4]:
rate1, data1 = wav.read("audio1.wav")
rate2, data2 = wav.read("audio2.wav")